PYTHON NOTEBOOK USED TO ANSWER TO EXERCISES OF CHAPTER 5 OF MATH80624 LECTURE NOTES (Solutions)

Modified by:
1. Chun Peng (Created for RSOME January 2021)
2. Erick Delage (January 2021)

As discussed in Chapter 5 of the  [lecture notes](http://web.hec.ca/pages/erick.delage/MATH80624_LectureNotes.pdf) of MATH80624 at HEC Montréal. 

WARNING!!!

The following code exploits a free Mosek licence for the course MATH80624 at HEC Montréal (expiration June 1st 2021). If you have error messages informing you about licencing issues, you may try uncommenting the installation lines for Gurobi. Otherwise, we recommend that you obtain your own licence of either Mosek ([url](https://www.mosek.com/)) or Gurobi ([url](https://www.gurobi.com/)).

**Jean-Sébastien Matte**

**Sena Onen Oz**

# **Preliminaries**

In [ ]:
!pip install rsome
!pip install mosek
!rm mosek.lic
!git clone https://github.com/erickdelage/80624
!cp ./80624/mosek.lic .
!rm -r ./80624
!mkdir -p /root/mosek
!cp ./mosek.lic /root/mosek
#!pip install -i https://pypi.gurobi.com gurobipy

     |████████████████████████████████| 10.1MB 5.1MB/s 
rm: cannot remove 'mosek.lic': No such file or directory
Cloning into '80624'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 2), reused 11 (delta 1), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [ ]:
import rsome as rso
import numpy as np
from rsome import ro
from rsome import msk_solver as my_solver  #Import Mosek solver interface
#from rsome import grb_solver as my_solver  #Import Gurobi solver interface


### Load the data

In [ ]:
n=4    #The number of facility locations
m=12   #The number of retailer locations
c = np.array([9.1, 8.0, 4.5, 2.1])                                                        #Installation cost for each facility location
r = 2*np.ones((n,m))                                                                      #Retalier price
P = np.array([23, 168, 110, 295])                                                         #Capacity of each facility locations
D_bar = np.array([24, 12, 18, 23, 24, 13, 11, 9, 18, 25, 25, 23])                         #Nominal demand of each retalier
D_hat = np.array([18, 1, 14, 12, 13, 5, 6, 0, 4, 23, 21, 20])                             #Maximum deviation from the nominal demand
d = np.array([[2.31, 2.37, 1.89, 1.92, 1.98, 1.69, 2.37, 2.14, 2.87, 2.16, 2.15, 1.52],   #Transportation cost from facility to retailers
    [1.88, 2.36, 2.02, 2.77, 1.17, 1.45, 2.64, 1.45, 1.83, 1.80, 1.74, 2.42],
    [2.51, 1.73, 3.50, 2.39, 2.51, 2.50, 3.08, 2.36, 2.35, 1.72, 1.47, 2.10],
    [1.71, 2.99, 1.40, 0.96, 1.79, 1.81, 1.89, 2.01, 2.28, 1.71, 2.98, 2.66]])

#Column constraint generation algorithm parameters
big_M = 10000
tolerance = 1e-6

# **Nominal facility location model**

We will study the following model:
\begin{align}
\max_{x,y} & - \sum_{i=1}^n c_i x_i + \sum_{i=1}^n \sum_{j=1}^m (r_{ij}-d_{ij})y_{ij} &&\\
\text{s.t.} & \sum_{i=1}^n y_{ij} \leq D_j &&\forall\,j \in\{1,\cdots,m\}\\
& \sum_{j=1}^m y_{ij} \leq P_i x_i &&\forall\,\;i \in\{1,\cdots,n\}\\
&  x \in \{0,1\}^n,\;y_{ij} \geq 0 &&\forall\,i \in\{1,\cdots,n\}, j \in\{1,\cdots,m\}, 
\end{align}


In [ ]:
#Solve the nominal facility location model

#Create model
model=ro.Model('Nominal facility location problem')
#Define variables
x = model.dvar(n,vtype='B')
y = model.dvar((n,m))          #in million of units

#List the objective and constraints
model.max(-c@x + ((r-d)*y).sum())
model.st(y.sum(axis=0) <= D_bar)       # Maximum demand at each retalier location
model.st(y.sum(axis=1) <= P*x)         # Capacity of each facility location
model.st(y>=0)                         

#Solve the model 
model.solve(my_solver)
opt_obj = model.get()  #in million of dollars
opt_x = np.round(x.get())

print('The objective is', opt_obj, 'and the optimal faciliy location is', opt_x)

Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0145s
The objective is 89.05 and the optimal faciliy location is [1. 1. 1. 1.]


# **Exercise 5.1) Implementing static RC**



\begin{align}
  \max_{x,y} \;\;&  - \sum_{i=1}^n c_i x_i + \sum_{i=1}^n \sum_{j=1}^m (r_{ij}-d_{ij})y_{ij} &&\\
  \text{s.t.} & \sum_{i=1}^n y_{ij} \leq \bar{D}_j+\hat{D}_j z_j &&\forall\,j \in\{1,\cdots,m\}, \; \forall \, z \in  \mathcal{Z}(\Gamma)\\
  & \sum_{j=1}^m y_{ij} \leq P_i x_i &&\forall\,\;i \in\{1,\cdots,n\}, \\
   & y_{ij} \geq 0 && \forall\,i \in\{1,\cdots,n\}, j \in\{1,\cdots,m\}, \\
  &  x \in \{0,1\}^n, && 
\end{align}

where \begin{align*}
            \mathcal{Z}(\Gamma):= \left\{ z\in \mathbb{R}^m\,\middle|\, -1 \leq z \leq 1,\, \sum_i |z_i| \leq \Gamma\right\}
        \end{align*}

In [ ]:
def FacilityLocation_SRC(n,m,D_bar,D_hat,r,P,c,d,Gamma):

    model=ro.Model('Static robust facility location problem')

    # Define uncertain factors
    z = model.rvar(m)
    budgetSet = (z<=1, z>=-1,  #each parameter is between [-1, 1] 
                      rso.norm(z,1)<=Gamma)   # Budget of uncertainty approach

    #Define variables
    x = model.dvar(n,vtype='B')
    y = model.dvar((n,m)) 

    #List the objective and constraints
    model.max(-c@x + ((r-d)*y).sum())
    model.st((y.sum(axis=0) <= D_bar+D_hat*z).forall(budgetSet))       # Maximum demand at each retalier location
    model.st(y.sum(axis=1) <= P*x)         # Capacity of each facility location
    model.st(y>=0)
    
    model.solve(my_solver)
    opt_obj_SRC = model.get()  #in million of dollars
    opt_x_SRC = np.round(x.get())
    return (opt_obj_SRC,opt_x_SRC)

#running an example
(opt_obj_SRC, opt_x_SRC) = FacilityLocation_SRC(n,m,D_bar,D_hat,r,P,c,d,1)
print('The objective is', opt_obj_SRC, 'and the optimal faciliy location is', opt_x_SRC)

Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0345s
The objective is 28.510000000000005 and the optimal faciliy location is [0. 1. 0. 1.]


# **Exercise 5.2) Implementing AARC**



\begin{align}
  \max_{x,y,Y}\min_{z} \;\;&  - \sum_{i=1}^n c_i x_i + \sum_{i=1}^n \sum_{j=1}^m (r_{ij}-d_{ij})(y_{ij}+Y_{ij}(z)) &&\\
  \text{s.t.} & \sum_{i=1}^n (y_{ij}+Y_{ij}(z)) \leq \bar{D}_j+\hat{D}_j z_j &&\forall\,j \in\{1,\cdots,m\}, \; \forall \, z \in  \mathcal{Z}(\Gamma)\\
  & \sum_{j=1}^m (y_{ij}+Y_{ij}(z)) \leq P_i x_i &&\forall\,\;i \in\{1,\cdots,n\},  \; \forall \, z \in  \mathcal{Z}(\Gamma) \\
   & (y_{ij}+Y_{ij}(z)) \geq 0 && \forall\,i \in\{1,\cdots,n\}, j \in\{1,\cdots,m\},  \; \forall \, z \in  \mathcal{Z}(\Gamma) \\
  &  x \in \{0,1\}^n, && 
\end{align}

where \begin{align*}
            \mathcal{Z}(\Gamma):= \left\{ z\in \mathbb{R}^m\,\middle|\, -1 \leq z \leq 1,\, \sum_i |z_i| \leq \Gamma\right\}
        \end{align*}

In [ ]:
def FacilityLocation_AARC(n,m,D_bar,D_hat,r,P,c,d,Gamma):
    model=ro.Model('AARC')

    # Define uncertain factors
    z = model.rvar(m)
    budgetSet = (z<=1, z>=-1,  #each parameter is between [-1, 1] 
                      rso.norm(z,1)<=Gamma)   # Budget of uncertainty approach

    x = model.dvar(n,vtype='B')
    y = model.ldr((n,m)) 

    # Define decision rules dependance
    y.adapt(z)

    #List the objective and constraints
    model.maxmin(-c@x + ((r-d)*y).sum(),budgetSet)
    model.st((y.sum(axis=0) <= D_bar+D_hat*z).forall(budgetSet))       # Maximum demand at each retalier location
    model.st((y.sum(axis=1) <= P*x).forall(budgetSet))      # Capacity of each facility location
    model.st(y>=0)
    
    model.solve(my_solver)
    opt_obj_AARC = model.get()  #in million of dollars
    opt_x_AARC = x.get()
    return (opt_obj_AARC, opt_x_AARC)

#running an example
(opt_obj_AARC, opt_x_AARC) = FacilityLocation_AARC(n,m,D_bar,D_hat,r,P,c,d,1)
print('The objective is', opt_obj_AARC, 'and the optimal faciliy location is', opt_x_AARC)

Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.9449s
The objective is 76.57000000000002 and the optimal faciliy location is [1. 1. 1. 1.]


# **Exercise 5.3 Implementing Lifted AARC**




\begin{align}
  \max_{x,y,Y}\min_{z} \;\;&  - \sum_{i=1}^n c_i x_i + \sum_{i=1}^n \sum_{j=1}^m (r_{ij}-d_{ij})(y_{ij}+Y_{ij}^{+}z^{+}+Y_{ij}^{-}z^{-}) &&\\
  \text{s.t.} & \sum_{i=1}^n (y_{ij}+Y_{ij}^{+}z^{+}+Y_{ij}^{-}z^{-}) \leq \bar{D}_j+\hat{D}_j z_j &&\forall\,j \in\{1,\cdots,m\}, \; \forall \, (z,z^{+}, z^{-}) \in  \mathcal{Z'}(\Gamma)\\
  & \sum_{j=1}^m (y_{ij}+Y_{ij}^{+}z^{+}+Y_{ij}^{-}z^{-}) \leq P_i x_i &&\forall\,\;i \in\{1,\cdots,n\},  \; \forall \, (z^{+}, z^{-}) \in  \mathcal{Z'}(\Gamma) \\
   & (y_{ij}+Y_{ij}^{+}z^{+}+Y_{ij}^{-}z^{-}) \geq 0 && \forall\,i \in\{1,\cdots,n\}, j \in\{1,\cdots,m\},  \; \forall \, (z^{+}, z^{-}) \in  \mathcal{Z'}(\Gamma) \\
  &  x \in \{0,1\}^n, && 
\end{align}

where \begin{align*}
            \mathcal{Z'}(\Gamma):= \left\{ (z,z^{+}, z^{-}) \in \mathbb{R}^m  \text{x} \mathbb{R}^m \text{x} \mathbb{R}^m\,\middle|\,z = z^{+} - z^{-}, \, z^{+} \ge 0, \, z^{-} \ge 0, \, z^{+} + z^{-} \leq 1 ,\,  \sum_i (z_{i}^{+} + z_{i}^{-}) \leq \Gamma\right\}
        \end{align*}

In [ ]:
#Define the function of solving LAARC model
def FacilityLocation_LAARC(n,m,D_bar,D_hat,r,P,c,d,Gamma):
    model=ro.Model('LAARC')

    z = model.rvar(m)
    zplus = model.rvar(m)
    zminus = model.rvar(m)
    budgetSet = (z==zplus-zminus, zplus>=0, zminus>=0, zplus+zminus<=1,  #each parameter is between [-1, 1] 
                      sum(zplus)+sum(zminus)<=Gamma)   # Budget of uncertainty approach

    x = model.dvar(n,vtype='B')
    y = model.ldr((n,m)) 

    # Define decision rules dependance
    y.adapt(zplus)
    y.adapt(zminus)

    #List the objective and constraints
    model.maxmin(-c@x + ((r-d)*y).sum(),budgetSet)
    model.st((y.sum(axis=0) <= D_bar+D_hat*z).forall(budgetSet))       # Maximum demand at each retalier location
    model.st((y.sum(axis=1) <= P*x).forall(budgetSet))      # Capacity of each facility location
    model.st(y>=0)

    model.solve(my_solver)
    opt_obj_LAARC = model.get()  #in million of dollars
    opt_x_LAARC = x.get()
    return (opt_obj_LAARC, opt_x_LAARC)

#running an example
(opt_obj_LAARC, opt_x_LAARC) = FacilityLocation_LAARC(n,m,D_bar,D_hat,r,P,c,d,1)
print('The objective is', opt_obj_LAARC, 'and the optimal faciliy location is', opt_x_LAARC)

Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.8346s
The objective is 76.57000000000002 and the optimal faciliy location is [1. 1. 1. 1.]


# **Exercise 5.4) Comparison of approximate worst-case bounds**

We wish to compare the different optimal values obtained from the three approximation models (RC, AARC, and LAARC)  on the robust facility location-transportation problem to the true worst-case value that can be achieved under the budgets $\Gamma\in\{0,\,1,\,4,\,m-1\}$.

### Prepare the exact solution scheme based on 4.3)

In [ ]:
def FacilityLocation_ColumnConstGen(n,m,D_bar,D_hat,r,P,c,d,Gamma,big_M,tolerance):

    # Define the MASTER model
    def master(z_m):
        master = ro.Model('master')

        #Define variables
        x_m = master.dvar(n, vtype='B')
        s_m = master.dvar(1)

        # Constraints
        for i in range(z_m.shape[1]):
          y_m = master.dvar((n, m))          #in million of units 
          master.st(s_m <= ((r-d)*y_m).sum())
          master.st(y_m.sum(axis=0) <= D_bar + (np.diag(D_hat) @ z_m[:,i])) # Maximum demand at each retalier location, for each vertex
          master.st(y_m.sum(axis=1) <= P * x_m)         # Capacity of each facility location
          master.st(y_m >= 0) 

        # objective function
        master.max(-c@x_m + s_m)                        

        #Solve the model 
        master.solve(my_solver)

        return master.get(), x_m.get()

    def slave(x_s):
        slave = ro.Model('slave')

        # Define variables
        z_s = slave.dvar(m)
        y_s = slave.dvar((n, m))
        lambda_s = slave.dvar(m)
        u_s = slave.dvar(m, vtype='B')
        gamma_s = slave.dvar(n)
        v_s = slave.dvar(n, vtype='B')
        theta_s = slave.dvar((n,m))
        w_s = slave.dvar((n,m), vtype='B')

        # objective function
        slave.min(-c@x_s + ((r-d)*y_s).sum())

        # Constraints
        slave.st(y_s.sum(axis=0) <= D_bar + (np.diag(D_hat) @ z_s)) 
        slave.st(y_s.sum(axis=1) <= P * x_s)   
        slave.st(-y_s <= 0)  
        for j in range(y_s.shape[0]):
          slave.st(lambda_s + (np.ones(m) * gamma_s[j]) - theta_s[j,:] == (r[j,:] - d[j,:]))
        slave.st(lambda_s >= 0) 
        slave.st(gamma_s >= 0)
        slave.st(theta_s >= 0)
        slave.st(lambda_s <= big_M*u_s) 
        slave.st(gamma_s <= big_M*v_s)
        slave.st(theta_s <= big_M*w_s)
        slave.st(D_bar + (np.diag(D_hat) @ z_s) - y_s.sum(axis=0) <= big_M*(np.ones(m)-u_s)) 
        slave.st((P * x_s) - y_s.sum(axis=1) <= big_M*(np.ones(n)-v_s))   
        slave.st(y_s <= big_M*(np.ones((n,m))-w_s)) 
        slave.st(abs(z_s) <= 1)
        slave.st(rso.norm(z_s, 1) <= Gamma)

        #Solve the model 
        slave.solve(my_solver)

        return slave.get(), z_s.get()

    # C&CG Algorithm
    x0 = [0, 0, 0, 0]

    _, z0 = slave(x0)

    Z_prime = np.array([[i] for i in z0])

    for iter in range(10):
        s_hat, x_hat = master(Z_prime)

        h_hat, z_hat = slave(x_hat)

        if abs(h_hat - s_hat) <= tolerance:
            opt_obj = h_hat 
            opt_x = x_hat 
            return (opt_obj, opt_x)
            break
        
        else:
            Z_prime = np.column_stack((Z_prime, np.array(z_hat)))


#running an example
(opt_obj_CCG, opt_x_CCG) = FacilityLocation_ColumnConstGen(n,m,D_bar,D_hat,r,P,c,d,1,big_M,tolerance)
print('The objective is', opt_obj_CCG, 'and the optimal faciliy location is', opt_x_CCG)

Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0099s
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0115s
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0119s
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0131s
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0123s
The objective is 76.57000000000002 and the optimal faciliy location is [1. 1. 1. 1.]


### Prepare a function evaluating the worst-case value for some x based on the slave problem in exercise 4.3)

In [ ]:
#solveWorstCaseProfitBudgetSet function
def solveWorstCaseProfitBudgetSet(x,n,m,c,r,P,D_bar,D_hat,d,big_M,Gamma):

    model = ro.Model('slave-problem')

    # Define variables
    z = model.dvar(m)
    y_s = model.dvar((n, m))
    lambda_s = model.dvar(m)
    u_s = model.dvar(m, vtype='B')
    gamma_s = model.dvar(n)
    v_s = model.dvar(n, vtype='B')
    theta_s = model.dvar((n,m))
    w_s = model.dvar((n,m), vtype='B')

    # objective function
    model.min(-c@x + ((r-d)*y_s).sum())

    # Constraints
    model.st(y_s.sum(axis=0) <= D_bar + (np.diag(D_hat) @ z)) 
    model.st(y_s.sum(axis=1) <= P * x)   
    model.st(-y_s <= 0)  
    for j in range(y_s.shape[0]):
      model.st(lambda_s + (np.ones(m) * gamma_s[j]) - theta_s[j,:] == (r[j,:] - d[j,:]))
    model.st(lambda_s >= 0) 
    model.st(gamma_s >= 0)
    model.st(theta_s >= 0)
    model.st(lambda_s <= big_M*u_s) 
    model.st(gamma_s <= big_M*v_s)
    model.st(theta_s <= big_M*w_s)
    model.st(D_bar + (np.diag(D_hat) @ z) - y_s.sum(axis=0) <= big_M*(np.ones(m)-u_s)) 
    model.st((P * x) - y_s.sum(axis=1) <= big_M*(np.ones(n)-v_s))   
    model.st(y_s <= big_M*(np.ones((n,m))-w_s)) 
    model.st(abs(z) <= 1)
    model.st(rso.norm(z, 1) <= Gamma)

    #Solve the model 
    model.solve(my_solver)

    fval = model.get()
    z = z.get()
    return (fval, z)

### Results

In [ ]:
#Solve the each type of conservative approximation model
results = ''
for Gamma in [0, 1, 4, m-1]:
    (bound1, opt_x1) = FacilityLocation_SRC(n,m,D_bar,D_hat,r,P,c,d,Gamma)
    (bound2, opt_x2) = FacilityLocation_AARC(n,m,D_bar,D_hat,r,P,c,d,Gamma)
    (bound3, opt_x3) = FacilityLocation_LAARC(n,m,D_bar,D_hat,r,P,c,d,Gamma)
    (bound4, opt_x4) = FacilityLocation_ColumnConstGen(n,m,D_bar,D_hat,r,P,c,d,Gamma,big_M,tolerance)
    
    (obj_RC, z_RC)= solveWorstCaseProfitBudgetSet(opt_x1,n,m,c,r,P,D_bar,D_hat,d,big_M,Gamma)
    (obj_AARC,x_AARC)= solveWorstCaseProfitBudgetSet(opt_x2,n,m,c,r,P,D_bar,D_hat,d,big_M,Gamma)
    (obj_LAARC,x_LAARC)= solveWorstCaseProfitBudgetSet(opt_x3,n,m,c,r,P,D_bar,D_hat,d,big_M,Gamma)
    (obj_CCG,x_CCG)= solveWorstCaseProfitBudgetSet(opt_x4,n,m,c,r,P,D_bar,D_hat,d,big_M,Gamma)
    
    results += 'When Gamma='+str(Gamma)+' the objectives of the four models are '+str(np.round([bound1, bound2, bound3, bound4], decimals=4))+'\n'
    results += '    while the worst-case profits of the policies are '+str(np.round([obj_RC, obj_AARC, obj_LAARC, obj_CCG], decimals=4))+'.\n'


Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0306s
Being solved by Mosek...
Solution status: integer_optimal
Running time: 1.0166s
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0851s
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0103s
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0166s
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0105s
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0105s
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0100s
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0101s
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0103s
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.0326s
Being solved by Mosek...
Solution status: integer_optimal
Running time: 0.9630s
Being solved by Mosek...
Solution status

In [ ]:
print(results)

When Gamma=0 the objectives of the four models are [89.05 89.05 89.05 89.05]
    while the worst-case profits of the policies are [89.05 89.05 89.05 89.05].
When Gamma=1 the objectives of the four models are [28.51 76.57 76.57 76.57]
    while the worst-case profits of the policies are [69.14 76.57 76.57 76.57].
When Gamma=4 the objectives of the four models are [28.51 44.31 45.05 45.05]
    while the worst-case profits of the policies are [43.28 44.31 45.05 45.05].
When Gamma=11 the objectives of the four models are [28.51 28.51 28.51 28.51]
    while the worst-case profits of the policies are [28.51 28.51 28.51 28.51].

